# Lab 4.3.5: Drift Detection with Evidently AI

**Module:** 4.3 - MLOps & Experiment Tracking  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand the types of drift (data, concept, prediction)
- [ ] Set up Evidently AI for drift monitoring
- [ ] Create data quality and drift reports
- [ ] Build a monitoring dashboard for production models
- [ ] Implement alerting for detected drift

---

## 📚 Prerequisites

- Completed: Lab 4.3.4 (Custom Evaluation)
- Knowledge of: Statistics basics, ML deployment concepts
- Hardware: DGX Spark (any configuration)

---

## 🌍 Real-World Context

**Your model works great today. But will it work next month?**

The world changes constantly:
- Customer behavior shifts (COVID changed everything!)
- New products/categories appear
- Seasonal patterns change
- Data pipelines break silently

**Real Drift Examples:**

| Company | What Happened | Impact |
|---------|---------------|--------|
| Zillow | Housing market shifted | $500M loss, shut down home-buying |
| Amazon | COVID changed shopping | Recommendation accuracy dropped |
| Banks | New fraud patterns | Missed fraud cases, customer losses |
| Healthcare | New COVID variants | Diagnostic models degraded |

**Drift monitoring is essential for production ML!**

---

## 🧒 ELI5: What is Model Drift?

> **Imagine you trained a dog to fetch tennis balls in your backyard.**
>
> The dog learned: "Yellow, round, bouncy = fetch!"
>
> Now imagine:
> - **Data drift**: You move to a new house with a different backyard
>   - The grass is artificial, the lighting is different
>   - The dog is confused: "This doesn't look like my training yard!"
>
> - **Concept drift**: You switch to orange baseballs
>   - The dog learned "yellow = fetch", but now the target is orange
>   - Same task, but the rules changed!
>
> - **Prediction drift**: The dog starts fetching sticks instead
>   - Something is wrong with its predictions
>
> **In ML:**
> - Data drift = input data distribution changes
> - Concept drift = relationship between inputs and outputs changes
> - Prediction drift = model outputs change unexpectedly

---

## Part 1: Understanding Drift Types

### The Three Types of Drift

| Type | What Changes | Example | Detection |
|------|-------------|---------|----------|
| **Data Drift** | Input distribution | Age distribution of users shifts | Compare input statistics |
| **Concept Drift** | Input→Output relationship | What "spam" means changes | Monitor prediction accuracy |
| **Prediction Drift** | Model outputs | More "Yes" predictions than usual | Track prediction distribution |

In [ ]:
# Install Evidently AI
import subprocess
import sys

try:
    import evidently
    print(f"✅ Evidently already installed: v{evidently.__version__}")
except ImportError:
    print("📦 Installing Evidently AI...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "evidently", "-q"])
    import evidently
    print(f"✅ Evidently installed: v{evidently.__version__}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pathlib import Path
import json

# Evidently imports
from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, DataQualityPreset, TargetDriftPreset
from evidently.metrics import (
    DatasetDriftMetric,
    ColumnDriftMetric,
    DatasetMissingValuesMetric,
    ColumnQuantileMetric
)

print(f"Evidently version: {evidently.__version__}")

In [ ]:
# Setup directories
NOTEBOOK_DIR = Path.cwd()
MODULE_DIR = (NOTEBOOK_DIR / "..").resolve()
REPORTS_DIR = MODULE_DIR / "evaluation" / "drift_reports"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 Reports will be saved to: {REPORTS_DIR}")

---

## Part 2: Creating Synthetic Data with Drift

Let's create realistic data that simulates drift over time.

In [ ]:
def generate_loan_data(n_samples: int, drift_factor: float = 0.0, seed: int = 42):
    """
    Generate synthetic loan application data.
    
    Args:
        n_samples: Number of samples to generate
        drift_factor: 0.0 = no drift, 1.0 = maximum drift
        seed: Random seed for reproducibility
    
    Returns:
        DataFrame with loan application features and predictions
    """
    np.random.seed(seed)
    
    # Base distributions
    age_mean = 35 + drift_factor * 10  # Ages shift older with drift
    income_mean = 50000 + drift_factor * 20000  # Incomes increase
    credit_score_mean = 680 - drift_factor * 50  # Credit scores decrease
    
    data = {
        "age": np.random.normal(age_mean, 10, n_samples).astype(int),
        "income": np.random.lognormal(np.log(income_mean), 0.5, n_samples),
        "credit_score": np.random.normal(credit_score_mean, 50, n_samples).astype(int),
        "debt_to_income": np.random.beta(2 + drift_factor, 5 - drift_factor, n_samples),
        "employment_years": np.random.exponential(5 - drift_factor * 2, n_samples),
        "loan_amount": np.random.lognormal(10 + drift_factor * 0.5, 0.8, n_samples),
        "num_credit_lines": np.random.poisson(3 + drift_factor * 2, n_samples),
    }
    
    df = pd.DataFrame(data)
    
    # Clip to reasonable ranges
    df["age"] = df["age"].clip(18, 80)
    df["credit_score"] = df["credit_score"].clip(300, 850)
    df["debt_to_income"] = df["debt_to_income"].clip(0, 1)
    df["employment_years"] = df["employment_years"].clip(0, 40)
    
    # Generate predictions (loan approval probability)
    # Model "learned" on original distribution
    base_prob = (
        0.3 + 
        0.002 * (df["credit_score"] - 600) +
        0.00001 * df["income"] -
        0.5 * df["debt_to_income"] +
        0.01 * df["employment_years"]
    )
    
    df["prediction_prob"] = base_prob.clip(0.05, 0.95)
    df["prediction"] = (df["prediction_prob"] > 0.5).astype(int)
    
    # Add actual outcome (with some noise)
    noise = np.random.uniform(-0.1, 0.1, n_samples)
    df["actual"] = ((base_prob + noise) > 0.5).astype(int)
    
    return df


# Generate reference (training) data and current (production) data
print("📊 Generating synthetic loan data...")

# Reference data (what the model was trained on)
reference_data = generate_loan_data(n_samples=5000, drift_factor=0.0, seed=42)
print(f"   Reference data: {len(reference_data)} samples")

# Current data with mild drift
current_data_mild = generate_loan_data(n_samples=1000, drift_factor=0.3, seed=100)
print(f"   Current data (mild drift): {len(current_data_mild)} samples")

# Current data with severe drift
current_data_severe = generate_loan_data(n_samples=1000, drift_factor=0.8, seed=200)
print(f"   Current data (severe drift): {len(current_data_severe)} samples")

In [ ]:
# Visualize the drift
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

features = ["age", "income", "credit_score", "debt_to_income", "employment_years", "loan_amount"]

for idx, feature in enumerate(features):
    ax = axes[idx // 3, idx % 3]
    
    ax.hist(reference_data[feature], bins=30, alpha=0.5, label="Reference", density=True)
    ax.hist(current_data_mild[feature], bins=30, alpha=0.5, label="Current (mild)", density=True)
    ax.hist(current_data_severe[feature], bins=30, alpha=0.5, label="Current (severe)", density=True)
    
    ax.set_xlabel(feature)
    ax.set_ylabel("Density")
    ax.set_title(f"Distribution: {feature}")
    ax.legend(fontsize=8)

plt.suptitle("Feature Distribution Drift Visualization", fontsize=14)
plt.tight_layout()
plt.savefig(REPORTS_DIR / "drift_visualization.png", dpi=150)
plt.show()

print(f"\n📊 Visualization saved to: {REPORTS_DIR / 'drift_visualization.png'}")

---

## Part 3: Creating Drift Reports with Evidently

Evidently makes it easy to create beautiful, interactive drift reports.

In [ ]:
# Define column mapping
# This tells Evidently which columns are features, targets, predictions, etc.

column_mapping = ColumnMapping(
    target="actual",
    prediction="prediction",
    numerical_features=[
        "age", "income", "credit_score", "debt_to_income",
        "employment_years", "loan_amount", "num_credit_lines"
    ]
)

print("✅ Column mapping configured")
print(f"   Target: {column_mapping.target}")
print(f"   Prediction: {column_mapping.prediction}")
print(f"   Numerical features: {len(column_mapping.numerical_features)}")

In [ ]:
# Create a comprehensive data drift report
print("📊 Creating Data Drift Report (mild drift)...")

drift_report_mild = Report(metrics=[
    DataDriftPreset(),
])

drift_report_mild.run(
    reference_data=reference_data,
    current_data=current_data_mild,
    column_mapping=column_mapping
)

# Save as HTML
mild_report_path = REPORTS_DIR / "drift_report_mild.html"
drift_report_mild.save_html(str(mild_report_path))
print(f"✅ Report saved to: {mild_report_path}")

In [ ]:
# Create report for severe drift
print("📊 Creating Data Drift Report (severe drift)...")

drift_report_severe = Report(metrics=[
    DataDriftPreset(),
])

drift_report_severe.run(
    reference_data=reference_data,
    current_data=current_data_severe,
    column_mapping=column_mapping
)

severe_report_path = REPORTS_DIR / "drift_report_severe.html"
drift_report_severe.save_html(str(severe_report_path))
print(f"✅ Report saved to: {severe_report_path}")

In [ ]:
# Extract drift metrics programmatically
def extract_drift_summary(report: Report) -> dict:
    """Extract key drift metrics from an Evidently report."""
    results = report.as_dict()
    
    summary = {
        "dataset_drift_detected": False,
        "drift_share": 0.0,
        "drifted_columns": [],
        "column_drift_scores": {}
    }
    
    # Parse results
    for metric in results.get("metrics", []):
        result = metric.get("result", {})
        
        if "dataset_drift" in result:
            summary["dataset_drift_detected"] = result.get("dataset_drift", False)
            summary["drift_share"] = result.get("share_of_drifted_columns", 0)
        
        if "drift_by_columns" in result:
            for col, col_result in result["drift_by_columns"].items():
                if col_result.get("drift_detected", False):
                    summary["drifted_columns"].append(col)
                summary["column_drift_scores"][col] = col_result.get("drift_score", 0)
    
    return summary


# Compare drift levels
print("\n📊 DRIFT COMPARISON")
print("=" * 60)

mild_summary = extract_drift_summary(drift_report_mild)
severe_summary = extract_drift_summary(drift_report_severe)

print(f"\n🟡 Mild Drift:")
print(f"   Dataset drift detected: {mild_summary['dataset_drift_detected']}")
print(f"   Drift share: {mild_summary['drift_share']:.1%}")
print(f"   Drifted columns: {mild_summary['drifted_columns']}")

print(f"\n🔴 Severe Drift:")
print(f"   Dataset drift detected: {severe_summary['dataset_drift_detected']}")
print(f"   Drift share: {severe_summary['drift_share']:.1%}")
print(f"   Drifted columns: {severe_summary['drifted_columns']}")

---

## Part 4: Data Quality Reports

Beyond drift, monitoring data quality is crucial.

In [ ]:
# Create data with quality issues
def add_quality_issues(df: pd.DataFrame, issue_rate: float = 0.1) -> pd.DataFrame:
    """
    Add data quality issues to a DataFrame.
    
    Args:
        df: Input DataFrame
        issue_rate: Fraction of values to corrupt
    
    Returns:
        DataFrame with quality issues
    """
    df_copy = df.copy()
    n = len(df_copy)
    
    # Add missing values
    for col in ["income", "employment_years"]:
        mask = np.random.random(n) < issue_rate
        df_copy.loc[mask, col] = np.nan
    
    # Add outliers
    mask = np.random.random(n) < issue_rate / 2
    df_copy.loc[mask, "age"] = np.random.choice([0, -5, 150, 200], mask.sum())
    
    # Add duplicates
    n_duplicates = int(n * issue_rate)
    duplicates = df_copy.sample(n=n_duplicates, replace=True)
    df_copy = pd.concat([df_copy, duplicates], ignore_index=True)
    
    return df_copy


# Create data with quality issues
current_with_issues = add_quality_issues(current_data_mild, issue_rate=0.15)
print(f"📊 Created data with quality issues: {len(current_with_issues)} samples")
print(f"   Missing values: {current_with_issues.isnull().sum().sum()}")
print(f"   Invalid ages: {(current_with_issues['age'] < 0).sum() + (current_with_issues['age'] > 120).sum()}")

In [ ]:
# Create data quality report
print("📊 Creating Data Quality Report...")

quality_report = Report(metrics=[
    DataQualityPreset(),
])

quality_report.run(
    reference_data=reference_data,
    current_data=current_with_issues,
    column_mapping=column_mapping
)

quality_report_path = REPORTS_DIR / "data_quality_report.html"
quality_report.save_html(str(quality_report_path))
print(f"✅ Report saved to: {quality_report_path}")

In [ ]:
# Extract quality metrics
quality_results = quality_report.as_dict()

print("\n📊 DATA QUALITY SUMMARY")
print("=" * 60)

for metric in quality_results.get("metrics", []):
    result = metric.get("result", {})
    
    if "current" in result:
        current = result["current"]
        if "number_of_missing_values" in current:
            print(f"\n📋 Current Data:")
            print(f"   Total rows: {current.get('number_of_rows', 'N/A')}")
            print(f"   Missing values: {current.get('number_of_missing_values', 0)}")
            print(f"   Duplicate rows: {current.get('number_of_duplicated_rows', 0)}")

---

## Part 5: Target and Prediction Drift

Monitor how predictions and actual outcomes change over time.

In [ ]:
# Create target drift report
print("📊 Creating Target Drift Report...")

target_drift_report = Report(metrics=[
    TargetDriftPreset(),
])

target_drift_report.run(
    reference_data=reference_data,
    current_data=current_data_severe,
    column_mapping=column_mapping
)

target_report_path = REPORTS_DIR / "target_drift_report.html"
target_drift_report.save_html(str(target_report_path))
print(f"✅ Report saved to: {target_report_path}")

In [ ]:
# Visualize prediction drift
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Prediction distribution
axes[0].hist(reference_data["prediction"], bins=3, alpha=0.5, label="Reference", density=True)
axes[0].hist(current_data_severe["prediction"], bins=3, alpha=0.5, label="Current (severe)", density=True)
axes[0].set_xlabel("Prediction")
axes[0].set_ylabel("Density")
axes[0].set_title("Prediction Distribution")
axes[0].legend()
axes[0].set_xticks([0, 1])
axes[0].set_xticklabels(["Rejected", "Approved"])

# Prediction probability distribution
axes[1].hist(reference_data["prediction_prob"], bins=30, alpha=0.5, label="Reference", density=True)
axes[1].hist(current_data_severe["prediction_prob"], bins=30, alpha=0.5, label="Current (severe)", density=True)
axes[1].set_xlabel("Approval Probability")
axes[1].set_ylabel("Density")
axes[1].set_title("Prediction Probability Distribution")
axes[1].legend()

plt.suptitle("Prediction Drift Analysis", fontsize=14)
plt.tight_layout()
plt.savefig(REPORTS_DIR / "prediction_drift.png", dpi=150)
plt.show()

---

## Part 6: Building a Monitoring System

Let's create a reusable monitoring system for production.

In [ ]:
from dataclasses import dataclass
from typing import Optional, List, Dict, Any
from datetime import datetime


@dataclass
class MonitoringConfig:
    """Configuration for drift monitoring."""
    drift_threshold: float = 0.5  # Fraction of drifted columns to alert
    quality_threshold: float = 0.05  # Max fraction of missing values
    alert_email: Optional[str] = None
    alert_slack_webhook: Optional[str] = None
    save_reports: bool = True
    report_dir: str = "./reports"


@dataclass
class MonitoringResult:
    """Result of a monitoring check."""
    timestamp: datetime
    drift_detected: bool
    drift_share: float
    drifted_columns: List[str]
    quality_issues: Dict[str, Any]
    alert_triggered: bool
    report_path: Optional[str] = None


class ProductionMonitor:
    """
    Production monitoring system for ML models.
    
    Tracks data drift, prediction drift, and data quality.
    """
    
    def __init__(self, reference_data: pd.DataFrame, column_mapping: ColumnMapping,
                 config: MonitoringConfig = None):
        self.reference_data = reference_data
        self.column_mapping = column_mapping
        self.config = config or MonitoringConfig()
        self.history: List[MonitoringResult] = []
        
        # Ensure report directory exists
        Path(self.config.report_dir).mkdir(parents=True, exist_ok=True)
    
    def check(self, current_data: pd.DataFrame, batch_id: str = None) -> MonitoringResult:
        """
        Run a monitoring check on current data.
        
        Args:
            current_data: Current production data
            batch_id: Optional identifier for this batch
        
        Returns:
            MonitoringResult with drift and quality analysis
        """
        timestamp = datetime.now()
        batch_id = batch_id or timestamp.strftime("%Y%m%d_%H%M%S")
        
        # Run drift detection
        drift_report = Report(metrics=[DataDriftPreset()])
        drift_report.run(
            reference_data=self.reference_data,
            current_data=current_data,
            column_mapping=self.column_mapping
        )
        
        drift_summary = extract_drift_summary(drift_report)
        
        # Check data quality
        quality_issues = {
            "missing_rate": current_data.isnull().mean().mean(),
            "columns_with_missing": current_data.columns[current_data.isnull().any()].tolist()
        }
        
        # Determine if alert should be triggered
        alert_triggered = (
            drift_summary["drift_share"] > self.config.drift_threshold or
            quality_issues["missing_rate"] > self.config.quality_threshold
        )
        
        # Save report if configured
        report_path = None
        if self.config.save_reports:
            report_path = str(Path(self.config.report_dir) / f"monitor_{batch_id}.html")
            drift_report.save_html(report_path)
        
        # Create result
        result = MonitoringResult(
            timestamp=timestamp,
            drift_detected=drift_summary["dataset_drift_detected"],
            drift_share=drift_summary["drift_share"],
            drifted_columns=drift_summary["drifted_columns"],
            quality_issues=quality_issues,
            alert_triggered=alert_triggered,
            report_path=report_path
        )
        
        # Store in history
        self.history.append(result)
        
        # Send alert if triggered
        if alert_triggered:
            self._send_alert(result)
        
        return result
    
    def _send_alert(self, result: MonitoringResult):
        """Send alert notification."""
        alert_message = f"""
🚨 ML MONITORING ALERT
=======================
Timestamp: {result.timestamp}
Drift Detected: {result.drift_detected}
Drift Share: {result.drift_share:.1%}
Drifted Columns: {result.drifted_columns}
Missing Rate: {result.quality_issues['missing_rate']:.1%}
Report: {result.report_path}
"""
        print(alert_message)
        
        # In production, you'd send to email/Slack here
        # if self.config.alert_email:
        #     send_email(self.config.alert_email, alert_message)
    
    def get_trend(self, metric: str = "drift_share", last_n: int = 10) -> List[float]:
        """Get trend of a metric over time."""
        return [getattr(r, metric) for r in self.history[-last_n:]]
    
    def plot_history(self):
        """Plot monitoring history."""
        if not self.history:
            print("No monitoring history available")
            return
        
        timestamps = [r.timestamp for r in self.history]
        drift_shares = [r.drift_share for r in self.history]
        missing_rates = [r.quality_issues["missing_rate"] for r in self.history]
        alerts = [r.alert_triggered for r in self.history]
        
        fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
        
        # Drift share over time
        axes[0].plot(timestamps, drift_shares, 'b-o', label='Drift Share')
        axes[0].axhline(y=self.config.drift_threshold, color='r', linestyle='--', label='Threshold')
        axes[0].fill_between(timestamps, 0, drift_shares, alpha=0.3)
        axes[0].set_ylabel('Drift Share')
        axes[0].set_title('Data Drift Over Time')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Mark alerts
        for i, (t, alert) in enumerate(zip(timestamps, alerts)):
            if alert:
                axes[0].axvline(x=t, color='red', alpha=0.3)
        
        # Missing rate over time
        axes[1].plot(timestamps, missing_rates, 'g-o', label='Missing Rate')
        axes[1].axhline(y=self.config.quality_threshold, color='r', linestyle='--', label='Threshold')
        axes[1].fill_between(timestamps, 0, missing_rates, alpha=0.3, color='green')
        axes[1].set_xlabel('Time')
        axes[1].set_ylabel('Missing Rate')
        axes[1].set_title('Data Quality Over Time')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()


print("✅ ProductionMonitor class defined")

In [ ]:
# Demo: Run the monitoring system
print("🔬 Running Production Monitoring Demo")
print("=" * 60)

# Initialize monitor
config = MonitoringConfig(
    drift_threshold=0.3,
    quality_threshold=0.05,
    report_dir=str(REPORTS_DIR)
)

monitor = ProductionMonitor(
    reference_data=reference_data,
    column_mapping=column_mapping,
    config=config
)

# Simulate multiple monitoring checks over time
drift_levels = [0.0, 0.1, 0.2, 0.3, 0.5, 0.7, 0.4, 0.3]

for i, drift in enumerate(drift_levels):
    # Generate data with varying drift
    current = generate_loan_data(n_samples=500, drift_factor=drift, seed=i*100)
    
    # Run monitoring check
    result = monitor.check(current, batch_id=f"batch_{i:03d}")
    
    status = "🚨" if result.alert_triggered else "✅"
    print(f"Batch {i}: drift={drift:.1f} -> {status} share={result.drift_share:.1%}")

In [ ]:
# Visualize monitoring history
monitor.plot_history()

---

## Part 7: Integrating with MLflow

Log monitoring results to MLflow for comprehensive tracking.

In [ ]:
import mlflow

# Setup MLflow
MLFLOW_DIR = MODULE_DIR / "mlflow"
mlflow.set_tracking_uri(f"file://{MLFLOW_DIR}")
mlflow.set_experiment("Model-Monitoring")

print(f"📊 MLflow tracking: {MLFLOW_DIR}")

In [ ]:
def log_monitoring_to_mlflow(result: MonitoringResult, model_name: str = "loan-classifier"):
    """
    Log monitoring results to MLflow.
    
    Args:
        result: MonitoringResult from a check
        model_name: Name of the model being monitored
    """
    with mlflow.start_run(run_name=f"monitor_{result.timestamp.strftime('%Y%m%d_%H%M%S')}"):
        # Log parameters
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("check_timestamp", result.timestamp.isoformat())
        
        # Log metrics
        mlflow.log_metric("drift_detected", int(result.drift_detected))
        mlflow.log_metric("drift_share", result.drift_share)
        mlflow.log_metric("num_drifted_columns", len(result.drifted_columns))
        mlflow.log_metric("missing_rate", result.quality_issues["missing_rate"])
        mlflow.log_metric("alert_triggered", int(result.alert_triggered))
        
        # Log drifted columns as param
        if result.drifted_columns:
            mlflow.log_param("drifted_columns", ", ".join(result.drifted_columns[:10]))
        
        # Log report as artifact
        if result.report_path:
            mlflow.log_artifact(result.report_path, artifact_path="drift_reports")
        
        # Set tags
        mlflow.set_tag("type", "monitoring")
        mlflow.set_tag("alert_status", "alert" if result.alert_triggered else "ok")


# Log our monitoring history
print("📊 Logging monitoring history to MLflow...")
for result in monitor.history:
    log_monitoring_to_mlflow(result)

print(f"✅ Logged {len(monitor.history)} monitoring checks")

---

## ✋ Try It Yourself: Exercise

**Task:** Create a monitoring system for your use case.

1. Create synthetic data representing your domain
2. Simulate drift scenarios relevant to your use case
3. Set up monitoring with appropriate thresholds
4. Run multiple checks and analyze trends
5. Log results to MLflow

<details>
<summary>💡 Hint</summary>

```python
# Example: E-commerce product recommendations
def generate_ecommerce_data(n_samples, drift_factor=0.0):
    return pd.DataFrame({
        "user_age": np.random.normal(35 + drift_factor*10, 10, n_samples),
        "session_duration": np.random.exponential(5 - drift_factor, n_samples),
        "items_viewed": np.random.poisson(10 + drift_factor*5, n_samples),
        "cart_value": np.random.lognormal(4 + drift_factor*0.5, 1, n_samples),
        "purchased": np.random.binomial(1, 0.3 + drift_factor*0.2, n_samples)
    })

# Create monitor with custom thresholds
config = MonitoringConfig(drift_threshold=0.4, quality_threshold=0.02)
```
</details>

In [ ]:
# YOUR CODE HERE

# Step 1: Create synthetic data


# Step 2: Simulate drift scenarios


# Step 3: Set up monitoring


# Step 4: Run checks and analyze


# Step 5: Log to MLflow


---

## ⚠️ Common Mistakes

### Mistake 1: Using Wrong Reference Data

In [ ]:
# ❌ WRONG: Using test data as reference
# reference = test_set  # This wasn't what the model was trained on!

# ✅ RIGHT: Use training data distribution
# reference = training_set  # Or a representative sample of it

print("Reference data should match what the model was trained on!")

### Mistake 2: Ignoring Seasonal Patterns

In [ ]:
# ❌ WRONG: Alerting on expected seasonal changes
# December shopping data will always differ from June!

# ✅ RIGHT: Use season-appropriate reference data
# reference_december = get_last_year_december_data()
# Or: Adjust thresholds for known seasonal periods

print("Consider seasonality when setting drift thresholds!")

### Mistake 3: Not Monitoring Prediction Distribution

In [ ]:
# ❌ WRONG: Only monitoring input features
# Features might look fine, but predictions could be all wrong!

# ✅ RIGHT: Monitor both inputs AND outputs
# - Input feature distributions
# - Prediction distributions  
# - Prediction confidence distributions
# - Actual outcome distributions (when available)

print("Monitor predictions AND inputs for complete coverage!")

---

## 🎉 Checkpoint

You've learned:
- ✅ Understanding data drift, concept drift, and prediction drift
- ✅ Creating drift reports with Evidently AI
- ✅ Monitoring data quality in production
- ✅ Building automated monitoring systems with alerting
- ✅ Integrating monitoring with MLflow

---

## 📖 Further Reading

- [Evidently AI Documentation](https://docs.evidentlyai.com/)
- [Data Drift in ML (Google)](https://developers.google.com/machine-learning/guides/rules-of-ml#rule_37_measure_training_serving_skew)
- [Concept Drift Paper](https://arxiv.org/abs/2004.05785)
- [WhyLogs for Monitoring](https://whylabs.ai/whylogs)

---

## 🧹 Cleanup

In [ ]:
import gc

plt.close('all')
gc.collect()

print(f"📁 Reports saved to: {REPORTS_DIR}")
print(f"📊 MLflow data saved to: {MLFLOW_DIR}")
print("\n✅ Resources cleaned up")

---

## 📝 Summary

In this lab, we:

1. **Learned** about data drift, concept drift, and prediction drift
2. **Created** drift and quality reports with Evidently AI
3. **Built** a production monitoring system with alerting
4. **Integrated** monitoring with MLflow for tracking
5. **Practiced** detecting and responding to drift scenarios

**Next up:** Lab 4.3.6 - Model Registry and Versioning!